# Pre-Processing of data

In [1]:
#importing the required libraries
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import re
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
stop=stopwords.words('english')
from nltk.stem.porter import PorterStemmer
from bs4 import BeautifulSoup


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data=pd.read_csv(r"C:\Users\asus\Documents\Projects\Question_Similarity_Pairs\Input\train.csv").sample(60000)

In [3]:
data.head(5)

,id,qid1,qid2,question1,question2,is_duplicate
217099,217099,323388,323389,How do I store a fingerprint in a browser?,Where should I store browser fingerprint?,0
296706,296706,268935,73790,Why people like dogs?,Why some people like dog?,1
237109,237109,226912,348198,Why do people choose to have children despite ...,Is it worthwhile bringing children into this w...,0
111142,111142,182115,182116,What are some advantages and disadvantages ofu...,What are the disadvantages of credit cards?,0
222983,222983,330686,330687,Are there minorities in the Hell's Angels?,How active are the Hell's Angels in Montréal?,0


In [4]:
data.dropna(axis=0,subset=['question1','question2'],how='any',inplace=True)

In [5]:
ques1_list=list(set(data['question1'].tolist()))
ques2_list=list(set(data['question2'].tolist()))
print("Unique question in question_1:",len(ques1_list))
print("Unique question in question_2:", len(ques2_list))
fin_list=ques1_list+ques2_list
print("Total % of Unique questions that we have in this corpus is :",len(fin_list)/(2*data.id.count()))

Unique question in question_1: 54515
Unique question in question_2: 54319
Total % of Unique questions that we have in this corpus is : 0.9069651160852681


In [6]:
def pre_process(x):
    #Normalization
    x=str(x).lower()
    #Removing Contraction
    x=x.replace("000,000","m").replace("000","k").replace("won't","will not").replace("can't","cannot").replace("shouldn't","should not")\
        .replace("didn't","did not").replace("doesn't","does not").replace("couldn't","could not").replace("ll","will").replace("%","percent")\
            .replace("$","dollar").replace("he's","he is").replace("he's","he is").replace("wouldn't","would not")
    #Removeing Unicode Characters
    x=re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "",x)
    #Removing Stopwords
    text=[word for word in x.split() if word  not in (stop)]
    #Stemming
    stemmer=PorterStemmer()
    text=" ".join([stemmer.stem(word) for word in text])
    #removing HTML tags
    bs4=BeautifulSoup(text)
    text=bs4.get_text()
    text=text.strip()
    return text


    
    


In [7]:
def cleaning(val):
    return data[val].apply(pre_process)

data['question1']=cleaning('question1')
data['question2']=cleaning('question2')

    

In [8]:
data.head(10)

,id,qid1,qid2,question1,question2,is_duplicate
217099,217099,323388,323389,store fingerprint browser,store browser fingerprint,0
296706,296706,268935,73790,peopl like dog,peopl like dog,1
237109,237109,226912,348198,peopl choos children despit world live,worthwhil bring children world given statist r...,0
111142,111142,182115,182116,advantag disadvantag ofupgrad credit card,disadvantag credit card,0
222983,222983,330686,330687,minor hewil angel,activ hewil angel montral,0
293777,293777,84905,147614,increas type speed fast,increas type accuraci speed,0
41435,41435,74814,74815,anim better death note code geass,better anim death note code geass,1
14447,14447,27667,27668,peopl think theyr attract,unattract peopl think attract overestim attract,1
68358,68358,118149,118150,rank score criteria use tc codevita season iv,rank score criteria use tc codevita season v,0
274849,274849,393527,393528,bad take one men daili multivitamin one day,take one pack multivitamin without doctor advic,0


In [9]:
data.to_csv(r"C:\Users\asus\Documents\Projects\Question_Similarity_Pairs\Input\preprocessed_data.csv",index=False)